# ETL Processes
Use this notebook to develop the ETL process for each of your tables before completing the `etl.py` file to load the whole datasets.

In [1]:
import os
import glob
import psycopg2
import pandas as pd
from sql_queries import *

In [2]:
conn = psycopg2.connect("host=127.0.0.1 dbname=sparkifydb user=student password=student")
cur = conn.cursor()

In [3]:
def get_files(filepath):
    all_files = []
    for root, dirs, files in os.walk(filepath):
        files = glob.glob(os.path.join(root,'*.json'))
        for f in files :
            all_files.append(os.path.abspath(f))
    
    return all_files

# Process `song_data`
In this first part, you'll perform ETL on the first dataset, `song_data`, to create the `songs` and `artists` dimensional tables.

Let's perform ETL on a single song file and load a single record into each table to start.
- Use the `get_files` function provided above to get a list of all song JSON files in `data/song_data`
- Select the first song in this list
- Read the song file and view the data

In [4]:
song_files = "data/song_data"

In [5]:
filepath = get_files(song_files)


In [6]:
df = pd.DataFrame()
for file in filepath:
    df= df.append(pd.read_json(file, lines=True), ignore_index=True)

In [7]:
df.head()

,artist_id,artist_latitude,artist_location,artist_longitude,artist_name,duration,num_songs,song_id,title,year
0,ARD7TVE1187B99BFB1,NaN,California - LA,NaN,Casual,218.93179,1,SOMZWCG12A8C13C480,I Didn't Mean To,0
1,ARNTLGG11E2835DDB9,NaN,,NaN,Clp,266.39628,1,SOUDSGM12AC9618304,Insatiable (Instrumental Version),0
2,AR8ZCNI1187B9A069B,NaN,,NaN,Planet P Project,269.81832,1,SOIAZJW12AB01853F1,Pink World,1984
3,AR10USD1187B99F3F1,NaN,"Burlington, Ontario, Canada",NaN,Tweeterfriendly Music,189.57016,1,SOHKNRJ12A6701D1F8,Drop of Rain,0
4,ARMJAGH1187FB546F3,35.14968,"Memphis, TN",-90.04892,The Box Tops,148.03546,1,SOCIWDW12A8C13D406,Soul Deep,1969


## #1: `songs` Table
#### Extract Data for Songs Table
- Select columns for song ID, title, artist ID, year, and duration
- Use `df.values` to select just the values from the dataframe
- Index to select the first (only) record in the dataframe
- Convert the array to a list and set it to `song_data`

In [8]:
# Get First row of data as list
#song_data = df[['song_id','title','artist_id','year','duration']].values[0].tolist()

# Get entire dataset
song_data = df[['song_id','title','artist_id','year','duration']]
song_data.head()

,song_id,title,artist_id,year,duration
0,SOMZWCG12A8C13C480,I Didn't Mean To,ARD7TVE1187B99BFB1,0,218.93179
1,SOUDSGM12AC9618304,Insatiable (Instrumental Version),ARNTLGG11E2835DDB9,0,266.39628
2,SOIAZJW12AB01853F1,Pink World,AR8ZCNI1187B9A069B,1984,269.81832
3,SOHKNRJ12A6701D1F8,Drop of Rain,AR10USD1187B99F3F1,0,189.57016
4,SOCIWDW12A8C13D406,Soul Deep,ARMJAGH1187FB546F3,1969,148.03546


#### Insert Record into Song Table
Implement the `song_table_insert` query in `sql_queries.py` and run the cell below to insert a record for this song into the `songs` table. Remember to run `create_tables.py` before running the cell below to ensure you've created/resetted the `songs` table in the sparkify database.

In [9]:
#cur.execute(song_table_insert, song_data)
#conn.commit()

for i, row in song_data.iterrows():
    cur.execute(song_table_insert, row)
    conn.commit()

In [10]:
try: 
    cur.execute("SELECT * FROM songs LIMIT 15;")
except psycopg2.Error as e: 
    print("Error: select *")
    print (e)

row = cur.fetchone()
while row:
   print(row)
   row = cur.fetchone()

('SOMZWCG12A8C13C480', "I Didn't Mean To", 'ARD7TVE1187B99BFB1', 0, Decimal('218.93179'))
('SOUDSGM12AC9618304', 'Insatiable (Instrumental Version)', 'ARNTLGG11E2835DDB9', 0, Decimal('266.39628'))
('SOIAZJW12AB01853F1', 'Pink World', 'AR8ZCNI1187B9A069B', 1984, Decimal('269.81832'))
('SOHKNRJ12A6701D1F8', 'Drop of Rain', 'AR10USD1187B99F3F1', 0, Decimal('189.57016'))
('SOCIWDW12A8C13D406', 'Soul Deep', 'ARMJAGH1187FB546F3', 1969, Decimal('148.03546'))
('SOYMRWW12A6D4FAB14', 'The Moon And I (Ordinary Day Album Version)', 'ARKFYS91187B98E58F', 0, Decimal('267.7024'))
('SOMJBYD12A6D4F8557', 'Keepin It Real (Skit)', 'ARD0S291187B9B7BF5', 0, Decimal('114.78159'))
('SOQHXMF12AB0182363', 'Young Boy Blues', 'ARGSJW91187B9B1D6B', 0, Decimal('218.77506'))
('SONHOTT12A8C13493C', 'Something Girls', 'AR7G5I41187FB4CE6C', 1982, Decimal('233.40363'))
('SOFSOCN12A8C143F5D', 'Face the Ashes', 'ARXR32B1187FB57099', 2007, Decimal('209.60608'))
('SOXVLOJ12AB0189215', 'Amor De Cabaret', 'ARKRRTF1187B9984DA

Run `test.ipynb` to see if you've successfully added a record to this table.

## #2: `artists` Table
#### Extract Data for Artists Table
- Select columns for artist ID, name, location, latitude, and longitude
- Use `df.values` to select just the values from the dataframe
- Index to select the first (only) record in the dataframe
- Convert the array to a list and set it to `artist_data`

In [11]:
# Get First row of data as list
#artist_data = df[['artist_id','artist_name','artist_location','artist_latitude','longitude']].values[0].tolist()

#Get Artist dataset
artist_data = df[['artist_id','artist_name','artist_location','artist_latitude','artist_longitude']]
artist_data.head()

,artist_id,artist_name,artist_location,artist_latitude,artist_longitude
0,ARD7TVE1187B99BFB1,Casual,California - LA,NaN,NaN
1,ARNTLGG11E2835DDB9,Clp,,NaN,NaN
2,AR8ZCNI1187B9A069B,Planet P Project,,NaN,NaN
3,AR10USD1187B99F3F1,Tweeterfriendly Music,"Burlington, Ontario, Canada",NaN,NaN
4,ARMJAGH1187FB546F3,The Box Tops,"Memphis, TN",35.14968,-90.04892


#### Insert Record into Artist Table
Implement the `artist_table_insert` query in `sql_queries.py` and run the cell below to insert a record for this song's artist into the `artists` table. Remember to run `create_tables.py` before running the cell below to ensure you've created/resetted the `artists` table in the sparkify database.

In [12]:
#cur.execute(artist_table_insert, artist_data)
#conn.commit()

for i, row in artist_data.iterrows():
    cur.execute(artist_table_insert, row)
    conn.commit()

In [13]:
try: 
    cur.execute("SELECT * FROM artists LIMIT 5;")
except psycopg2.Error as e: 
    print("Error: select *")
    print (e)

row = cur.fetchone()
while row:
   print(row)
   row = cur.fetchone()

('ARD7TVE1187B99BFB1', 'Casual', 'California - LA', Decimal('NaN'), Decimal('NaN'))
('ARNTLGG11E2835DDB9', 'Clp', '', Decimal('NaN'), Decimal('NaN'))
('AR8ZCNI1187B9A069B', 'Planet P Project', '', Decimal('NaN'), Decimal('NaN'))
('AR10USD1187B99F3F1', 'Tweeterfriendly Music', 'Burlington, Ontario, Canada', Decimal('NaN'), Decimal('NaN'))
('ARMJAGH1187FB546F3', 'The Box Tops', 'Memphis, TN', Decimal('35.14968'), Decimal('-90.04892'))


In [14]:
#cur.close()
#conn.close()

Run `test.ipynb` to see if you've successfully added a record to this table.

# Process `log_data`
In this part, you'll perform ETL on the second dataset, `log_data`, to create the `time` and `users` dimensional tables, as well as the `songplays` fact table.

Let's perform ETL on a single log file and load a single record into each table.
- Use the `get_files` function provided above to get a list of all log JSON files in `data/log_data`
- Select the first log file in this list
- Read the log file and view the data

In [15]:
log_files = "data/log_data"

In [16]:
filepath = get_files(log_files)

In [17]:
df = pd.DataFrame()
for file in filepath:
    df= df.append(pd.read_json(file, lines=True), ignore_index=True)
df.head()

,artist,auth,firstName,gender,itemInSession,lastName,length,level,location,method,page,registration,sessionId,song,status,ts,userAgent,userId
0,Stephen Lynch,Logged In,Jayden,M,0,Bell,182.85669,free,"Dallas-Fort Worth-Arlington, TX",PUT,NextSong,1.540992e+12,829,Jim Henson's Dead,200,1543537327796,Mozilla/5.0 (compatible; MSIE 10.0; Windows NT...,91
1,Manowar,Logged In,Jacob,M,0,Klein,247.56200,paid,"Tampa-St. Petersburg-Clearwater, FL",PUT,NextSong,1.540558e+12,1049,Shell Shock,200,1543540121796,"""Mozilla/5.0 (Macintosh; Intel Mac OS X 10_9_4...",73
2,Morcheeba,Logged In,Jacob,M,1,Klein,257.41016,paid,"Tampa-St. Petersburg-Clearwater, FL",PUT,NextSong,1.540558e+12,1049,Women Lose Weight (Feat: Slick Rick),200,1543540368796,"""Mozilla/5.0 (Macintosh; Intel Mac OS X 10_9_4...",73
3,Maroon 5,Logged In,Jacob,M,2,Klein,231.23546,paid,"Tampa-St. Petersburg-Clearwater, FL",PUT,NextSong,1.540558e+12,1049,Won't Go Home Without You,200,1543540625796,"""Mozilla/5.0 (Macintosh; Intel Mac OS X 10_9_4...",73
4,Train,Logged In,Jacob,M,3,Klein,216.76363,paid,"Tampa-St. Petersburg-Clearwater, FL",PUT,NextSong,1.540558e+12,1049,Hey_ Soul Sister,200,1543540856796,"""Mozilla/5.0 (Macintosh; Intel Mac OS X 10_9_4...",73


## #3: `time` Table
#### Extract Data for Time Table
- Filter records by `NextSong` action
- Convert the `ts` timestamp column to datetime
  - Hint: the current timestamp is in milliseconds
- Extract the timestamp, hour, day, week of year, month, year, and weekday from the `ts` column and set `time_data` to a list containing these values in order
  - Hint: use pandas' [`dt` attribute](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.Series.dt.html) to access easily datetimelike properties.
- Specify labels for these columns and set to `column_labels`
- Create a dataframe, `time_df,` containing the time data for this file by combining `column_labels` and `time_data` into a dictionary and converting this into a dataframe

In [18]:
df = df.loc[df['page'] == 'NextSong']
df.head()

,artist,auth,firstName,gender,itemInSession,lastName,length,level,location,method,page,registration,sessionId,song,status,ts,userAgent,userId
0,Stephen Lynch,Logged In,Jayden,M,0,Bell,182.85669,free,"Dallas-Fort Worth-Arlington, TX",PUT,NextSong,1.540992e+12,829,Jim Henson's Dead,200,1543537327796,Mozilla/5.0 (compatible; MSIE 10.0; Windows NT...,91
1,Manowar,Logged In,Jacob,M,0,Klein,247.56200,paid,"Tampa-St. Petersburg-Clearwater, FL",PUT,NextSong,1.540558e+12,1049,Shell Shock,200,1543540121796,"""Mozilla/5.0 (Macintosh; Intel Mac OS X 10_9_4...",73
2,Morcheeba,Logged In,Jacob,M,1,Klein,257.41016,paid,"Tampa-St. Petersburg-Clearwater, FL",PUT,NextSong,1.540558e+12,1049,Women Lose Weight (Feat: Slick Rick),200,1543540368796,"""Mozilla/5.0 (Macintosh; Intel Mac OS X 10_9_4...",73
3,Maroon 5,Logged In,Jacob,M,2,Klein,231.23546,paid,"Tampa-St. Petersburg-Clearwater, FL",PUT,NextSong,1.540558e+12,1049,Won't Go Home Without You,200,1543540625796,"""Mozilla/5.0 (Macintosh; Intel Mac OS X 10_9_4...",73
4,Train,Logged In,Jacob,M,3,Klein,216.76363,paid,"Tampa-St. Petersburg-Clearwater, FL",PUT,NextSong,1.540558e+12,1049,Hey_ Soul Sister,200,1543540856796,"""Mozilla/5.0 (Macintosh; Intel Mac OS X 10_9_4...",73


In [19]:
df['date'] =  pd.to_datetime(df['ts']) ## pandas recognizes your format
df['time'] = df['date'].dt.time
df['hour'] = df['date'].dt.hour
df['day'] = df['date'].dt.day
df['weekofyear'] = df['date'].dt.weekofyear
df['month'] = df['date'].dt.month
df['year_from_ts'] = df['date'].dt.year
df['weekday'] = df['date'].dt.weekday
t = df[['ts','date','time','hour','day','weekofyear','month','year_from_ts','weekday']]
t.head()

,ts,date,time,hour,day,weekofyear,month,year_from_ts,weekday
0,1543537327796,1970-01-01 00:25:43.537327796,00:25:43.537327,0,1,1,1,1970,3
1,1543540121796,1970-01-01 00:25:43.540121796,00:25:43.540121,0,1,1,1,1970,3
2,1543540368796,1970-01-01 00:25:43.540368796,00:25:43.540368,0,1,1,1,1970,3
3,1543540625796,1970-01-01 00:25:43.540625796,00:25:43.540625,0,1,1,1,1970,3
4,1543540856796,1970-01-01 00:25:43.540856796,00:25:43.540856,0,1,1,1,1970,3


In [20]:
#time_data = (t.values[0].tolist())
#column_labels = ()

In [21]:
time_df = t[['date','hour','day','weekofyear','month','year_from_ts','weekday']]
time_df.head()

,date,hour,day,weekofyear,month,year_from_ts,weekday
0,1970-01-01 00:25:43.537327796,0,1,1,1,1970,3
1,1970-01-01 00:25:43.540121796,0,1,1,1,1970,3
2,1970-01-01 00:25:43.540368796,0,1,1,1,1970,3
3,1970-01-01 00:25:43.540625796,0,1,1,1,1970,3
4,1970-01-01 00:25:43.540856796,0,1,1,1,1970,3


#### Insert Records into Time Table
Implement the `time_table_insert` query in `sql_queries.py` and run the cell below to insert records for the timestamps in this log file into the `time` table. Remember to run `create_tables.py` before running the cell below to ensure you've created/resetted the `time` table in the sparkify database.

In [22]:
for i, row in time_df.iterrows():
    cur.execute(time_table_insert, list(row))
    conn.commit()

In [23]:
try: 
    cur.execute("SELECT * FROM time LIMIT 5;")
except psycopg2.Error as e: 
    print("Error: select *")
    print (e)

row = cur.fetchone()
while row:
   print(row)
   row = cur.fetchone()

(datetime.time(0, 25, 43, 537328), 0, 1, 1, 1, 1970, 3)
(datetime.time(0, 25, 43, 540122), 0, 1, 1, 1, 1970, 3)
(datetime.time(0, 25, 43, 540369), 0, 1, 1, 1, 1970, 3)
(datetime.time(0, 25, 43, 540626), 0, 1, 1, 1, 1970, 3)
(datetime.time(0, 25, 43, 540857), 0, 1, 1, 1, 1970, 3)


Run `test.ipynb` to see if you've successfully added records to this table.

## #4: `users` Table
#### Extract Data for Users Table
- Select columns for user ID, first name, last name, gender and level and set to `user_df`

In [24]:
user_df = df[['userId','firstName','lastName','gender','level']]
user_df.head()

,userId,firstName,lastName,gender,level
0,91,Jayden,Bell,M,free
1,73,Jacob,Klein,M,paid
2,73,Jacob,Klein,M,paid
3,73,Jacob,Klein,M,paid
4,73,Jacob,Klein,M,paid


#### Insert Records into Users Table
Implement the `user_table_insert` query in `sql_queries.py` and run the cell below to insert records for the users in this log file into the `users` table. Remember to run `create_tables.py` before running the cell below to ensure you've created/resetted the `users` table in the sparkify database.

In [25]:
for i, row in user_df.iterrows():
    cur.execute(user_table_insert, row)
    conn.commit()

In [26]:
try: 
    cur.execute("SELECT * FROM users LIMIT 5;")
except psycopg2.Error as e: 
    print("Error: select *")
    print (e)

row = cur.fetchone()
while row:
   print(row)
   row = cur.fetchone()

(57, 'Katherine', 'Gay', 'F', 'free')
(86, 'Aiden', 'Hess', 'M', 'free')
(74, 'Braden', 'Parker', 'M', 'free')
(92, 'Ryann', 'Smith', 'F', 'free')
(73, 'Jacob', 'Klein', 'M', 'paid')


Run `test.ipynb` to see if you've successfully added records to this table.

## #5: `songplays` Table
#### Extract Data and Songplays Table
This one is a little more complicated since information from the songs table, artists table, and original log file are all needed for the `songplays` table. Since the log file does not specify an ID for either the song or the artist, you'll need to get the song ID and artist ID by querying the songs and artists tables to find matches based on song title, artist name, and song duration time.
- Implement the `song_select` query in `sql_queries.py` to find the song ID and artist ID based on the title, artist name, and duration of a song.
- Select the timestamp, user ID, level, song ID, artist ID, session ID, location, and user agent and set to `songplay_data`

#### Insert Records into Songplays Table
- Implement the `songplay_table_insert` query and run the cell below to insert records for the songplay actions in this log file into the `songplays` table. Remember to run `create_tables.py` before running the cell below to ensure you've created/resetted the `songplays` table in the sparkify database.

In [27]:
try: 
    cur.execute("select songs.artist_id, \
                songs.song_id, songs.title, \
                artists.name,artists.location, \
                songs.duration \
                FROM songs INNER JOIN artists \
                ON songs.artist_id = artists.artist_id LIMIT 5;")
except psycopg2.Error as e: 
    print("Error: select *")
    print (e)

row = cur.fetchone()
while row:
   print(row)
   row = cur.fetchone()

('ARD7TVE1187B99BFB1', 'SOMZWCG12A8C13C480', "I Didn't Mean To", 'Casual', 'California - LA', Decimal('218.93179'))
('ARD7TVE1187B99BFB1', 'SOMZWCG12A8C13C480', "I Didn't Mean To", 'Casual', 'California - LA', Decimal('218.93179'))
('ARD7TVE1187B99BFB1', 'SOMZWCG12A8C13C480', "I Didn't Mean To", 'Casual', 'California - LA', Decimal('218.93179'))
('ARD7TVE1187B99BFB1', 'SOMZWCG12A8C13C480', "I Didn't Mean To", 'Casual', 'California - LA', Decimal('218.93179'))
('ARD7TVE1187B99BFB1', 'SOMZWCG12A8C13C480', "I Didn't Mean To", 'Casual', 'California - LA', Decimal('218.93179'))


In [28]:
df[['ts','time','userId','level','sessionId','location','userAgent','artist','length', 'song']].head()

,ts,time,userId,level,sessionId,location,userAgent,artist,length,song
0,1543537327796,00:25:43.537327,91,free,829,"Dallas-Fort Worth-Arlington, TX",Mozilla/5.0 (compatible; MSIE 10.0; Windows NT...,Stephen Lynch,182.85669,Jim Henson's Dead
1,1543540121796,00:25:43.540121,73,paid,1049,"Tampa-St. Petersburg-Clearwater, FL","""Mozilla/5.0 (Macintosh; Intel Mac OS X 10_9_4...",Manowar,247.56200,Shell Shock
2,1543540368796,00:25:43.540368,73,paid,1049,"Tampa-St. Petersburg-Clearwater, FL","""Mozilla/5.0 (Macintosh; Intel Mac OS X 10_9_4...",Morcheeba,257.41016,Women Lose Weight (Feat: Slick Rick)
3,1543540625796,00:25:43.540625,73,paid,1049,"Tampa-St. Petersburg-Clearwater, FL","""Mozilla/5.0 (Macintosh; Intel Mac OS X 10_9_4...",Maroon 5,231.23546,Won't Go Home Without You
4,1543540856796,00:25:43.540856,73,paid,1049,"Tampa-St. Petersburg-Clearwater, FL","""Mozilla/5.0 (Macintosh; Intel Mac OS X 10_9_4...",Train,216.76363,Hey_ Soul Sister


In [29]:
import random
import string
''.join(random.choice(string.ascii_uppercase + string.digits) for _ in range(16))

'P7DY5QBBHBYMJ0WX'

In [30]:
for index, row in df.iterrows():

    # get songid and artistid from song and artist tables
    cur.execute(song_select, (row.song, row.artist, row.length))
    results = cur.fetchone()
    
    if results:
        songid, artistid = results
    else:
        songid, artistid = None, None
        
    # insert songplay record
    songplay_data = (row.time,row.userId, row.level,songid, artistid,row.sessionId, row.location
                     ,row.userAgent
                    )
    cur.execute(songplay_table_insert, songplay_data)
    conn.commit()

In [31]:
try: 
    cur.execute("SELECT * FROM songplays WHERE song_id is NOT NULL and artist_id is NOT NULL LIMIT 5;")
except psycopg2.Error as e: 
    print("Error: select *")
    print (e)

row = cur.fetchone()
while row:
   print(row)
   row = cur.fetchone()

(4108, datetime.time(0, 25, 42, 837407), '15', 'paid', 'SOZCTXZ12AB0182364', 'AR5KOSW1187FB35FF4', 818, 'Chicago-Naperville-Elgin, IL-IN-WI', '"Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Ubuntu Chromium/36.0.1985.125 Chrome/36.0.1985.125 Safari/537.36"')
(10928, datetime.time(0, 25, 42, 837407), '15', 'paid', 'SOZCTXZ12AB0182364', 'AR5KOSW1187FB35FF4', 818, 'Chicago-Naperville-Elgin, IL-IN-WI', '"Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Ubuntu Chromium/36.0.1985.125 Chrome/36.0.1985.125 Safari/537.36"')


Run `test.ipynb` to see if you've successfully added records to this table.

# Close Connection to Sparkify Database

In [32]:
conn.close()

# Implement `etl.py`
Use what you've completed in this notebook to implement `etl.py`.